# Código do Trabalho Final do Curso BI Master - PUC-Rio

### Utilizando HugginFace - Pytorch

Alterar o notebook no colab (ou Kaggle) para que possa utilizar GPU afim de melhorar o desempenho de preprocessamento, treinamento e executação.

In [ ]:
# Instação das bibliotecas utilizadas
!pip install --upgrade pip
!pip install transformers
!pip install unidecode
!pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab,faiss]

In [ ]:
## Importação das bibliotecas
import pandas as pd
import numpy as np
import string
import unidecode
import os, glob, re, sys, random, unicodedata, collections

import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

plt.style.use('ggplot')

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.feature_extraction.text import CountVectorizer

from tqdm import tqdm
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer
from nltk.tokenize import sent_tokenize , word_tokenize
nltk.download('words')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('vader_lexicon')
nltk.download('stopwords')
## criando lista de stopwords
STOP_WORDS = stopwords.words('portuguese') + list(string.punctuation)
STOP_WORDS.append('\n')

## Importação do BERT - Hugginface
from transformers import AutoModel, AutoTokenizer, BertTokenizer, BertForSequenceClassification
import torch

## Bibliotecas para busca em PT

## Bibliotecas para buscas em EN (apenas para exibição mais didática)
from haystack.utils import clean_wiki_text, convert_files_to_docs, fetch_archive_from_http, print_answers
from haystack.nodes import FARMReader, TransformersReader, DensePassageRetriever, PreProcessor
from haystack.document_stores import FAISSDocumentStore
from haystack.pipelines import ExtractiveQAPipeline


/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


INFO - haystack.modeling.model.optimization -  apex not found, won't use it. See https://nvidia.github.io/apex/
ERROR - root -  Failed to import 'magic' (from 'python-magic' and 'python-magic-bin' on Windows). FileTypeClassifier will not perform mimetype detection on extensionless files. Please make sure the necessary OS libraries are installed if you need this functionality.


In [ ]:
## Montagem do Drive e Ponteiramento para o diretório
from google.colab import drive
drive.mount('/content/drive')

work_path = '/content/drive/MyDrive/Classroom/PosGraduacao/ProjetoFinal/'
os.chdir(work_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
## Instanciando os livros em uma lista local para realização dos futuros tratamentos
random.seed(50)

files = []

for dirname, _, filenames in os.walk('data/machado-de-assis/raw/txt'):
  for filename in filenames:
    files.append(os.path.join(dirname, filename))

print('There are a total of {} files'.format(len(files)), '\n')
files[:10]


There are a total of 116 files 



In [ ]:
# Seleção de 10 arquivos aleatórios
sample_books = random.sample(files,10)

docs = []
for fname in sample_books:
    with open(fname , "r") as file:
        text = file.read()
    docs.append(text)

# Contagem do Termo de Frequência (TF) utilizando CountVectorizer do scikit-learn
## limitando o número de palavras apenas para apresentação do conceito
vec = CountVectorizer(max_features=10, stop_words=STOP_WORDS) 
X = vec.fit_transform(docs)
df = pd.DataFrame(X.toarray(), columns=vec.get_feature_names_out()) #get_feature_names()) <- método em vias de obsolescência
books_names = [book.split('/')[-1] for book in sample_books]
df['book'] = books_names
df = df.set_index('book')

print(df)

                           ainda  aqui  assim  homem  outra  outro  pode  ser  \
book                                                                            
idealCritico.txt               2     0      1      2      1      3     8   11   
discursosAcademia.txt          6     0      0      1      1      1     2    6   
americanas.txt                 8     7     21     14      6     13     7    7   
revistaTeatros.txt            62    41     55     21     24     22    26   30   
passadoPresenteFuturo.txt      1     1      6      2      1      2     4    7   
semana.txt                   629   421    454    471    437    475   530  705   
licaoBotanica.txt              4     4      5      8      3      7    17   25   
eduardo.txt                    1     1      3      2      0      2     0    0   
magalhaes.txt                  4     7      3      1      1      5     1    0   
cartaBispoRJ.txt               1     1      2      0      0      0     2    5   

                           

In [ ]:
document_store = FAISSDocumentStore(faiss_index_factory_str="Flat")

INFO - haystack.telemetry -  Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by calling disable_telemetry() or by manually setting the environment variable HAYSTACK_TELEMETRY_ENABLED as described for different operating systems on the documentation page. More information at https://haystack.deepset.ai/guides/telemetry


In [ ]:
# Apontamento para o diretório onde encontram-se os arquivo que serão utilizados
doc_dir = "data/machado-de-assis/raw/txt"
# Conversão dos arquivos em dicionários
docs = convert_files_to_docs(dir_path=doc_dir, clean_func=clean_wiki_text, split_paragraphs=True)

# Now, let's write the dicts containing documents to our DB.
document_store.write_documents(docs)

Os códigos abaixo estão em vias de melhorias para tratamento e processamento dos dados.
Já possível verificar que há um retorno dos dados porém sem um resultado plausível uma vez que o modelo utilizado está baseado no idioma inglês.

Serão realizados testes com o modelo de DPR fornecido pela [Haystack](https://haystack.deepset.ai/overview/intro) e com o modelo de Q&A em PT afim de verificar qual atenderá melhor o projeto

In [ ]:
# Em teste para operações no idioma PT do preprocessamento
preprocessor = PreProcessor(language="pt")

In [ ]:
retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
    passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
    max_seq_len_query=64,
    max_seq_len_passage=256,
    batch_size=16,
    use_gpu=True,
    embed_title=True,
    use_fast_tokenizers=True,
)
# Important:
# Now that after we have the DPR initialized, we need to call update_embeddings() to iterate over all
# previously indexed documents and update their embedding representation.
# While this can be a time consuming operation (depending on corpus size), it only needs to be done once.
# At query time, we only need to embed the query and compare it the existing doc embeddings which is very fast.
document_store.update_embeddings(retriever)

INFO - haystack.modeling.utils -  Using devices: CPU
INFO - haystack.modeling.utils -  Number of GPUs: 0


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/493 [00:00<?, ?B/s]

...

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizerFast'.
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find facebook/dpr-ctx_encoder-single-nq-base locally.
INFO - haystack.modeling.model.language_model -  Looking on Transformers Model Hub (in local cache and online)...


Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

INFO - haystack.modeling.model.language_model -  Loaded facebook/dpr-ctx_encoder-single-nq-base
INFO - haystack.document_stores.faiss -  Updating embeddings for 116 docs...


Updating Embedding:   0%|          | 0/116 [00:00<?, ? docs/s]

Create embeddings:   0%|          | 0/128 [00:00<?, ? Docs/s]

In [ ]:
#reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)
reader = FARMReader(model_name_or_path="pierreguillou/bert-base-cased-squad-v1.1-portuguese", use_gpu=True)

INFO - haystack.modeling.utils -  Using devices: CPU
INFO - haystack.modeling.utils -  Number of GPUs: 0
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find pierreguillou/bert-base-cased-squad-v1.1-portuguese locally.
INFO - haystack.modeling.model.language_model -  Looking on Transformers Model Hub (in local cache and online)...


Downloading:   0%|          | 0.00/862 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/413M [00:00<?, ?B/s]

INFO - haystack.modeling.model.language_model -  Loaded pierreguillou/bert-base-cased-squad-v1.1-portuguese


Downloading:   0%|          | 0.00/494 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/205k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

INFO - haystack.modeling.utils -  Using devices: CPU
INFO - haystack.modeling.utils -  Number of GPUs: 0
INFO - haystack.modeling.infer -  Got ya 2 parallel workers to do inference ...
INFO - haystack.modeling.infer -   0     0  
INFO - haystack.modeling.infer -  /w\   /w\ 
INFO - haystack.modeling.infer -  /'\   / \ 


In [ ]:
pipe = ExtractiveQAPipeline(reader, retriever)

In [ ]:
# You can configure how many candidates the reader and retriever shall return
# The higher top_k for retriever, the better (but also the slower) your answers.
prediction = pipe.run(
    query="Quem é Capitu?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
)

Inferencing Samples:   0%|          | 0/7 [00:00<?, ? Batches/s]/usr/local/lib/python3.7/dist-packages/haystack/modeling/model/prediction_head.py:483: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  start_indices = flat_sorted_indices // max_seq_len
Inferencing Samples: 100%|██████████| 1/1 [00:06<00:00,  6.77s/ Batches]


In [ ]:
print_answers(prediction, details="medium")


Query: Quem é Capitu?
Answers:
[   {   'answer': 'um quadragenário\norgulhoso',
        'context': 'ar-nos.\n'
                   'João da Fonseca teve um primeiro gesto de acordo; era um '
                   'quadragenário\n'
                   'orgulhoso, para quem tal proposta era de si mesma uma '
                   'ofensa. Durante u',
        'score': 0.7946575880050659},
    {   'answer': 'quase minha irmã',
        'context': 'LIA — Por desgraça minha; mas há coisa pior...\n'
                   'EMÍLIA — Escuta; és quase minha irmã; não te posso ocultar '
                   'nada.\n'
                   'EMÍLIA — Isto mesmo. Amo-o doidamente,',
        'score': 0.7421748042106628},
    {   'answer': 'um\nente animado',
        'context': 'ava, sorria, e o vidro exprimia tudo. Não era mais um '
                   'autômato, era um\n'
                   'ente animado. Daí em diante, fui outro. Cada dia, a uma '
                   'certa hora, vestia-me d',
        'score': 0.6852

# Até o momento os código abaixo não foram utilizados

In [ ]:
# BERT Base
tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')
model = AutoModel.from_pretrained('neuralmind/bert-base-portuguese-cased')

#tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')
#model = BertForSequenceClassification.from_pretrained('neuralmind/bert-base-portuguese-cased')

# BERT Large
#tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-large-portuguese-cased')
#model = AutoModel.from_pretrained('neuralmind/bert-large-portuguese-cased')

#tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-large-portuguese-cased')
#model = BertForSequenceClassification.from_pretrained('neuralmind/bert-large-portuguese-cased')